# MNIST数据集训练评估
数据集有60 000个样本，每个图片都是28像素*28像素大小。每个像素将是训练时的特征值。

使用全连接神经网络对MNIST数据集处理

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
print("number of train data is %d"%(mnist.train.num_examples))
print("number of test data is %d"%(mnist.test.num_examples))
trainimg=mnist.train.images
trainlabel=mnist.train.labels
testimg=mnist.test.images
testlabel=mnist.test.labels

W1112 22:03:58.187894  4624 deprecation.py:323] From <ipython-input-1-d0bc52f92c82>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1112 22:03:58.191919  4624 deprecation.py:323] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1112 22:03:58.196871  4624 deprecation.py:323] From D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
number of train data is 55000
number of test data is 10000


### 构建神经网络
定义神经网络模型，神经网络模型定义可以分四步，构建输入层、隐层、输出层和定义损失函数。

对于MNIST数据集来说，输入的是每一个图片文件，维度为28*28=784，定义如下
x_=tf.placeholder(tf.float32,[None,784])

对于隐层，将首先根据神经元节点的对应连接线的权重和偏置项进行如下处理：
$ y=权重\times x+偏置项 $
然后经过激活函数的非线性化处理，得到输出向量。

神经网络模型构造如下

In [3]:
def add_layer(input,in_size,out_size,activation_function):
    Weight=tf.Variable(tf.random_normal([in_size,out_size]))
    biases=tf.Variable(tf.zeros([1,out_size]))
    Wx_plus_b=tf.matmul(input,Weight)+biases
    if activation_function is None:
        outputs=Wx_plus_b
    else:
        outputs=activation_function(Wx_plus_b)
    return outputs

- 在本例中，不使用隐层，直接从输入层到输出层。
- 对于输出层，最终输出MNIST数据集中的0~9共10个分类，输出值的维度为10。
- 对于损失函数，一般选取softmax交叉熵损失函数：
tf.nn.softmax_cross_entropy_with_logits(logits,labels,name=None)

其中，参数logits是输出层的输出向量，参数labels就是数据实际的标签值。
softmax操作将逻辑回归中预测二分类的概率问题推广到n分类概率问题。

In [4]:
x_=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])
# 没有隐层隐层
predition=add_layer(x_,784,10,activation_function=None)
# 损失函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=predition))
train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

W1112 22:15:50.557821  4624 deprecation.py:323] From <ipython-input-4-40b2d2c1a483>:6: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 进行数据训练
总共迭代101次，每训练5次，输出一次损失值
### 模型评估
可以使用tf.grgmax(y,1)方法获取根据任意输入x预测到的标记值。将该值与实际值匹配，
预测值转化为浮点数，取平均值得准确率。具体实现如下：

In [10]:
training_epochs=101
# 每次迭代选择100个样本
batch_size=100
display_step=5
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        # 训练过程
        avg_cost=0.
        num_batch=int(mnist.train.num_examples/batch_size)
        for i in range(num_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,{x_:batch_xs,y_:batch_ys})
            avg_cost+=sess.run(cross_entropy,{x_:batch_xs,y_:batch_ys})/num_batch
        # 训练到一定程度后，用模型预测测试数据
        if epoch%display_step==0:
            correct_prediction=tf.equal(tf.argmax(predition,1),tf.argmax(y_,1))
            accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
            test_acc=sess.run(accuracy,
                              feed_dict={x_:mnist.test.images,
                                         y_:mnist.test.labels})
            print("Epoch:%d/%d cost:%f TEST ACCURACY:%f"%(epoch,
                                                          training_epochs,
                                                          avg_cost,
                                                          test_acc))
            

Epoch:0/101 cost:1.059508 TEST ACCURACY:0.854800
Epoch:5/101 cost:0.339368 TEST ACCURACY:0.894300
Epoch:10/101 cost:0.282913 TEST ACCURACY:0.908200
Epoch:15/101 cost:0.259557 TEST ACCURACY:0.913200
Epoch:20/101 cost:0.244758 TEST ACCURACY:0.915300
Epoch:25/101 cost:0.234983 TEST ACCURACY:0.918300
Epoch:30/101 cost:0.225801 TEST ACCURACY:0.919700
Epoch:35/101 cost:0.221336 TEST ACCURACY:0.920400
Epoch:40/101 cost:0.217146 TEST ACCURACY:0.920200
Epoch:45/101 cost:0.213415 TEST ACCURACY:0.919600
Epoch:50/101 cost:0.211046 TEST ACCURACY:0.924000
Epoch:55/101 cost:0.211933 TEST ACCURACY:0.920200
Epoch:60/101 cost:0.209318 TEST ACCURACY:0.920500
Epoch:65/101 cost:0.207227 TEST ACCURACY:0.922800
Epoch:70/101 cost:0.205804 TEST ACCURACY:0.918700
Epoch:75/101 cost:0.202467 TEST ACCURACY:0.917600
Epoch:80/101 cost:0.200223 TEST ACCURACY:0.922700
Epoch:85/101 cost:0.206777 TEST ACCURACY:0.923800
Epoch:90/101 cost:0.199127 TEST ACCURACY:0.919600
Epoch:95/101 cost:0.198192 TEST ACCURACY:0.921900
Ep

### 亦可以使用多层神经网络模型
对于隐层，假设有256个神经元，使用relu函数作为激活函数，具体修改如下：

In [ ]:
# 使用一个隐层
n_input=784
n_hidden_1=256
n_classes=10
l1=add_layer(x_,n_input,n_hidden_1,activation_function=tf.nn.relu)
predition=add_layer(x_,n_hidden_1,n_classes,activation_function=None)